In [ ]:
!pip install -q catboost
!pip install -q SpeechRecognition
!pip install -q wavio
!pip install -q pydub
#!pip install -q --upgrade google-api-python-client
#!pip install -q google-colab

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import write as wav_write
import io
import speech_recognition as sr
import wavio
from os import path
from pydub import AudioSegment
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import write as wav_write
import io
import pandas as pd
import re
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from google.colab import drive
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
#drive.mount('/content/drive')

<a id='1'></a>
# Data Load

In [ ]:
df = pd.read_csv("/content/magaza_yorumlari.csv",encoding = "utf-16")
df.head()

<a id='2'></a>
# Data Preprocessing

In [ ]:
sns.countplot(x = df.Durum);

In [ ]:
sent_dict = {'Olumsuz': 0, 'Olumlu': 1}
df['Durum'] = df['Durum'].map(sent_dict)
df.head()

In [ ]:
def clean_text(text):

    unwanted_pattern = r'[!.\n,:“”,?@#"]'
    regex = re.compile(unwanted_pattern)
    cleaned_text = regex.sub(" ", text)
    
    return cleaned_text

df['new_text'] = df['Görüş'].astype(str).apply(clean_text)
df['new_text'] = df['new_text'].str.replace("[\d]", "")
df['new_text'] = df['new_text'].str.lower()
df

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ineffective = stopwords.words('turkish')
ineffective[:5]

In [ ]:
df['new_text'] = df['new_text'].apply(lambda x: " ".join(x for x in x.split() if x not in ineffective))
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['new_text'], df['Durum'], test_size=0.2, random_state=42)

tf_vector = TfidfVectorizer(max_features =  5000)
ct_vector = CountVectorizer(max_features = 5000)
#X = tf_vector.fit_transform(df['new_text'])
#Y = df['sentiment']
X_train_vec = tf_vector.fit_transform(X_train)
X_test_vec = tf_vector.transform(X_test)
X_train_ct = ct_vector.fit_transform(X_train)
X_test_ct = ct_vector.transform(X_test)

<a id='3'></a>
# Machine Learning Model

<a id='3.2'></a>
### Support Vector Machine

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, cbar=False, fmt=".4g")
    plt.show()
    print(classification_report(y_test, y_pred))
    
result = pd.DataFrame(columns=['TF_Vectorizer_accuracy', 'Count_Vectorizer_accuracy', 'Model'])

In [ ]:
svc_tf = SVC()
svc_tf.fit(X_train_vec, y_train)
evaluate_model(svc_tf, X_test_vec, y_test)

In [ ]:
svc_ct = SVC()
svc_ct.fit(X_train_ct, y_train)
evaluate_model(svc_ct, X_test_ct, y_test)
result = result.append({
                'Model': 'Support Vector Machine',
                'Count_Vectorizer_accuracy':svc_ct .score(X_test_ct, y_test),
                'TF_Vectorizer_accuracy': svc_tf.score(X_test_vec, y_test)},
                ignore_index=True)
result

In [ ]:
# files                                                                         
src = "/content/common_voice_tr_36558001.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

In [ ]:
#recognizer sınıfının tanıtılması
sesi_tani = sr.Recognizer()

# %%
#Kayıdın okutulması
with sr.AudioFile('/content/test.wav') as kaynak:

    ses_metni = sesi_tani.listen(kaynak)

    try:
        # Google ses tanıma servisini kullan
        textedvoice = sesi_tani.recognize_google(ses_metni, language = "tr-TR")
        print('Ses metni:')
        print(textedvoice)

    except:
         print('Bir hata oluştu, üzgünüm.')

In [ ]:
predict_dict = {0: 'Olumsuz', 1: 'Olumlu'}

text = str(textedvoice)

ml_token = tf_vector.transform([text])
pred_ml = svc_tf.predict(ml_token)[0]

print(f'Makine Öğrenmesi Tahmini / Machine Learning Predict = > {predict_dict[pred_ml]}')
